# Amazon SageMaker MLOps를 통한 머신 러닝 운영화

이 일련의 노트북은 간단한 노트북에서 ML 아이디어를 개발하는 것부터 자동화된 모델 구축 및 CI/CD 배포 파이프라인이 있는 프로덕션 솔루션까지 안내합니다.

실습은 다음과 같은 논리적 단계로 구성되어 있습니다:
1. SageMaker Studio Jupyterlab 노트북에서 데이터 수집 및 ML 모델 구축
2. SageMaker Pipelines의 비주얼 에디터 기능을 사용한 ML 파이프라인 오케스트레이션
3. 모델 구축 CI/CD 파이프라인 추가
4. 모델 배포 CI/CD 파이프라인 추가

![](img/sagemaker-mlops-diagram.jpg)

또한 [A/B 테스팅](https://docs.aws.amazon.com/sagemaker/latest/dg/model-validation.html), 커스텀 [처리](https://docs.aws.amazon.com/sagemaker/latest/dg/build-your-own-processing-container.html), [훈련](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-training-algo.html) 및 [추론](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-inference-main.html) 컨테이너, [디버깅 및 프로파일링](https://docs.aws.amazon.com/sagemaker/latest/dg/train-debugger.html), [보안](https://docs.aws.amazon.com/sagemaker/latest/dg/security.html), [다중 모델](https://docs.aws.amazon.com/sagemaker/latest/dg/multi-model-endpoints.html) 및 [다중 컨테이너](https://docs.aws.amazon.com/sagemaker/latest/dg/multi-container-endpoints.html) 엔드포인트, [직렬 추론 파이프라인](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-pipelines.html)과 같은 다른 SageMaker 기능과 ML 주제에 대한 추가 실습 예제도 있습니다. 이러한 기능을 테스트하려면 `additional-topics` 폴더의 노트북을 살펴보세요.

이 노트북과 워크샵의 모든 노트북을 실행하려면 JupyterLab에서 `Python 3` 커널을 사용하세요.

## 설정
SageMaker Python SDK의 최신 버전을 가져옵니다.

<div class="alert alert-info"> 💡 워크샵과 모든 노트북은 SageMaker Python SDK(sagemaker 패키지) 버전 2.219.0에서 테스트되었습니다. 노트북에서는 sagemaker의 버전을 고정하지 않습니다. 호환성 문제가 발생하면 다음 pip 명령을 실행하여 특정 버전의 sagemaker를 설치할 수 있습니다: <code>%pip install sagemaker=2.219.0</code>
</div>

In [ ]:
# Uncomment if you have any compatibility issues and would like to use the specific version of the sagemaker library
%pip install sagemaker==2.219.0
# %pip install --upgrade pip sagemaker boto3

In [ ]:
%pip install mlflow==2.13.2 sagemaker-mlflow

### 패키지 가져오기

In [ ]:
import time
import os
import json
import boto3
import numpy as np  
import pandas as pd 
import sagemaker
from time import gmtime, strftime, sleep

(sagemaker.__version__,boto3.__version__)

In [ ]:
import time
import os
import json
import boto3
import numpy as np  
import pandas as pd 
import sagemaker
from time import gmtime, strftime, sleep

(sagemaker.__version__,boto3.__version__)

In [ ]:
import json
def write_params(step_name, params):
    with open(f"{step_name}.json", "w") as f:    
        f.write(json.dumps(params))

def read_params(step_name):
    with open(f"{step_name}.json", "r") as f:
        data = f.read()
        params = json.loads(data)
    return params

### 변수 인스턴스화

In [ ]:
# Get some variables you need to interact with SageMaker service
bucket_name = sagemaker.Session().default_bucket()
sm_role = sagemaker.get_execution_role()
print(sm_role)

### MLflow 추적 서버에 연결
AWS 주도 워크샵을 실행 중이거나 제공된 CloudFormation 템플릿을 사용하여 워크샵 환경을 프로비저닝한 경우, MLflow 서버가 실행 중이어야 합니다. MLflow 서버가 없는 경우 [개발자 가이드](https://docs.aws.amazon.com/sagemaker/latest/dg/mlflow-create-tracking-server.html)를 따르거나 다음 코드 셀을 실행하여 새 서버를 생성하세요.

MLflow 추적 서버를 생성하고 관리하며 관리형 MLflow 실험을 사용하려면 SageMaker 실행 역할에 다음 권한이 연결되어 있어야 합니다:

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "sagemaker-mlflow:*",
                "sagemaker:CreateMlflowTrackingServer",
                "sagemaker:UpdateMlflowTrackingServer",
                "sagemaker:DeleteMlflowTrackingServer",
                "sagemaker:StartMlflowTrackingServer",
                "sagemaker:StopMlflowTrackingServer",
                "sagemaker:CreatePresignedMlflowTrackingServerUrl"
            ],
            "Resource": "*"
        }
    ]
}
```

실행 중인 MLflow 서버가 있는지 확인하려면 다음 코드를 실행하세요.

In [ ]:
# Find an active MLflow server in the account
r = boto3.client("sagemaker").list_mlflow_tracking_servers(
    TrackingServerStatus='Created',
)['TrackingServerSummaries']

if len(r) < 1:
    print("You don't have any running MLflow servers. Trying to find a server in the status 'Creating'...")

    r = boto3.client("sagemaker").list_mlflow_tracking_servers(
        TrackingServerStatus='Creating',
    )['TrackingServerSummaries']

    if len(r) < 1:
        print("You don't have any MLflow server in the status 'Creating'. Run the next code cell to create a new one.")
        mlflow_arn = None
        mlflow_name = None
    else:
        mlflow_arn = r[0]['TrackingServerArn']
        mlflow_name = r[0]['TrackingServerName']
        print(f"You have an MLflow server {mlflow_arn} in the status 'Creating', going to use this one")
else:
    mlflow_arn = r[0]['TrackingServerArn']
    mlflow_name = r[0]['TrackingServerName']
    print(f"You have {len(r)} running MLflow server(s). Get the first server ARN:{mlflow_arn}")

In [ ]:
# This code cell creates a new MLflow server
if not mlflow_arn:
    ts = strftime('%d-%H-%M-%S', gmtime())
    mlflow_name = f"mlflow-{ts}"
    r = boto3.client("sagemaker").create_mlflow_tracking_server(
        TrackingServerName=mlflow_name,
        ArtifactStoreUri=f"s3://{bucket_name}/mlflow/{ts}",
        RoleArn=sm_role,
        AutomaticModelRegistration=True,
    )

    mlflow_arn = r['TrackingServerArn']
    print(f"Server creation request succeded. The server {mlflow_arn} is being created.")

<div style="border: 4px solid coral; text-align: center; margin: auto;">
MLflow 서버 생성에는 최대 25분이 소요될 수 있습니다. 기다릴 필요 없이 워크샵을 계속 진행하세요.
</div>

In [ ]:
(mlflow_arn, mlflow_name)

# 요약 및 다음 단계
이 노트북에서는 워크샵을 성공적으로 실행하는 데 필요한 인프라를 검증하는 단계를 수행했습니다.